<a href="https://colab.research.google.com/github/BlackDeViL1906/MLOps_Learning/blob/main/MLOps_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "zenml[server]"
!zenml integration install sklearn -y
!pip install pyparsing===2.4.2
!pip install sqlalchemy>=2.0

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)


NumExpr defaulting to 2 threads.
⠙ Installing integrations...


{'status': 'ok', 'restart': True}

In [1]:
NGROK_Token = "2hmjEmnxI6RIQx5230pmkzRDgXF_4uwaLyXhiVHQaWhqDxhit"

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab():
    !pip install pyngrok
    from pyngrok import ngrok
    !ngrok authtoken $NGROK_Token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.58.2
⠋ Initializing ZenML repository at /content.
Creating database tables
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
Creating default workspace 'default' ...
Creating default stack in workspace default...
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML reposi

In [4]:
import numpy as np
import pandas as pd
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def train_test() -> None:

  """Train and test a Scikit-learn SVC classifier on digits"""
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split( data, digits.target, test_size=0.2, shuffle=False )

  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc =  model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")

train_test()

Test accuracy: 0.9583333333333334


In [5]:
from re import X
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "Y_train"],
    Annotated[np.ndarray, "Y_test"],
]:
  """Load the digits dataset as numpy arrays."""
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, Y_train, Y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
  return X_train, X_test, Y_train, Y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    Y_train: np.ndarray,
) -> ClassifierMixin:
  """Train a Scikit-learn SVC classifier."""
  model = SVC(gamma=0.001)
  model.fit(X_train, Y_train)
  return model

@step
def svc_evaluator(
    model: ClassifierMixin,
    X_test: np.ndarray,
    Y_test: np.ndarray,
) -> float:
  """Evaluate the trained Scikit-learn SVC classifier."""
  test_acc = model.score(X_test, Y_test)
  print(f"Test Accuracy: {test_acc}")
  return test_acc

In [6]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  """Links all the steps together in a pipeline."""
  X_train, X_test, Y_train, Y_test = importer()
  model = svc_trainer(X_train, Y_train)
  test_acc = svc_evaluator(model, X_test, Y_test)

In [7]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Step importer has started.
Step importer has finished in 3.053s.
Step importer completed successfully.
Step svc_trainer has started.
Step svc_trainer has finished in 0.690s.
Step svc_trainer completed successfully.
Step svc_evaluator has started.
Test Accuracy: 0.9583333333333334
Step svc_evaluator has finished in 0.944s.
Step svc_evaluator completed successfully.
Pipeline run has finished in 5.269s.


In [12]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok
        ngrok.set_auth_token('2hmjEmnxI6RIQx5230pmkzRDgXF_4uwaLyXhiVHQaWhqDxhit')  # Ensure you set your ngrok auth token here

        public_url = ngrok.connect(port)
        print(f"\033[1;32mIn Colab, use this URL instead: {public_url}!\033[0m")
        get_ipython().system(f"zenml up --blocking --port {port}")
    else:
        get_ipython().system(f"zenml up --port {port}")

start_zenml_dashboard()


INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.10/dist-packages/pyngrok/bin/ngrok


Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.10/dist-packages/pyngrok/bin/ngrok


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-024dbe60-54ea-479a-b585-ec6af8fad095


Opening tunnel named: http-8237-024dbe60-54ea-479a-b585-ec6af8fad095


INFO:pyngrok.process.ngrok:t=2024-06-12T16:46:11+0000 lvl=info msg="no configuration paths supplied"


...

In Colab, use this URL instead: NgrokTunnel: "https://50a8-34-73-1-7.ngrok-free.app" -> "http://localhost:8237"!
t=2024-06-12T16:46:12+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-024dbe60-54ea-479a-b585-ec6af8fad095 addr=http://localhost:8237 url=https://50a8-34-73-1-7.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2024-06-12T16:46:12+0000 lvl=info msg=end pg=/api/tunnels id=2d48907544681093 status=201 dur=43.980083ms


t=2024-06-12T16:46:12+0000 lvl=info msg=end pg=/api/tunnels id=2d48907544681093 status=201 dur=43.980083ms
NumExpr defaulting to 2 threads.
The local ZenML dashboard is about to deploy in a blocking process. You can connect to it using the 
'default' username and an empty password.
Deploying a local ZenML server with name 'local'.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [22213]
INFO:     Waiting for application startup.
Not writing the global configuration to disk in a ZenML server environment.
Not writing the global configuration to disk in a ZenML server environment.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Press CTRL+C to quit)


INFO:pyngrok.process.ngrok:t=2024-06-12T17:00:11+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2024-06-12T17:00:11+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:     Shutting down
INFO:     Finished server process [22213]
ERROR:    Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
  File "uvloop/loop.pyx", line 474, in uvloop.loop.Loop._on_idle
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.pyx", line 63, in uvloop.loop.Handle._run
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 68, in serve
    with self.capture_signals():
  File "/usr/lib/python3.10/contextlib.py", line 142, in __exit__
    next(